In [45]:
import pandas as pd
import json
import os
import ast
import glob

In [35]:
path = "/Users/srikarsamudrala/Desktop/Files/Project/Stern/data/Home_and_Kitchen.json" 

df = pd.read_json(path, lines=True)

In [36]:
# def parse_helpful(x):
#     try:
#         pair = ast.literal_eval(str(x))
#         return pair if len(pair)==2 else [0,0]
#     except:
#         return [0,0]

# df[['helpful_votes', 'total_votes']] = df['helpful'].apply(lambda x: pd.Series(parse_helpful(x)))

In [37]:
df = df.rename(columns={
    'reviewText': 'review',
    'summary': 'summary',
    'class': 'is_spam',
})

df = df.drop_duplicates(subset=['review'])
df = df.dropna(subset=['review', 'is_spam'])
df = df[df['review'].str.strip() != ""]

In [38]:
cols_to_drop = ['_id', 'unixReviewTime', 'reviewTime']

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

df['asin'] = df['asin'].astype('category')
df['category'] = df['category'].astype('category')

df = df.sort_values(by="asin").reset_index(drop=True)
df.head()

,reviewerID,asin,reviewerName,helpful,review,overall,summary,category,is_spam
0,A2PAVURT4NOHE1,0000031852,Leah,"[0, 0]",Bought it for a ballet tutu but it is being wo...,5,Super cute,Sports_and_Outdoors,1
1,A1SNLWGLFXD70K,0000031852,DEVA,"[0, 0]",I origonally didn't get the item I ordered. W...,4,Happy with purchase even though it came a lot ...,Sports_and_Outdoors,1
2,A3URQ0LXLV46E9,0000031852,shortyvee,"[0, 0]",My daughter and her friends love the colors an...,4,zebralisous,Sports_and_Outdoors,1
3,A1KJ4CVG87QW09,0000031852,Donna Carter-Scott,"[0, 0]","Arrived very timely, cute grandbaby loves it. ...",4,Cute Tutu,Sports_and_Outdoors,1
4,AA9ITO6ZLZW6,0000031852,Jazzy77,"[0, 0]",My little girl just loves to wear this tutu be...,5,Versatile,Sports_and_Outdoors,1


In [39]:
import re

def clean_for_sentiment(t):
    t = str(t).lower()
    t = re.sub(r"http\S+", "", t)  # remove URLs
    return t  # keep punctuation & emojis

df['clean_review'] = df['review'].apply(clean_for_sentiment)
df['clean_summary'] = df['summary'].apply(clean_for_sentiment)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007446 entries, 0 to 3007445
Data columns (total 11 columns):
 #   Column         Dtype   
---  ------         -----   
 0   reviewerID     object  
 1   asin           category
 2   reviewerName   object  
 3   helpful        object  
 4   review         object  
 5   overall        int64   
 6   summary        object  
 7   category       category
 8   is_spam        int64   
 9   clean_review   object  
 10  clean_summary  object  
dtypes: category(2), int64(2), object(7)
memory usage: 240.5+ MB


In [41]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Count of spam vs non-spam
# sns.countplot(x='is_spam', data=df)
# plt.title('Distribution of Spam vs Non-Spam Reviews')
# plt.show()

# # Percentage
# print(df['is_spam'].value_counts(normalize=True) * 100)

In [42]:
unique_asin_count = df['asin'].nunique()
print(f"Number of unique ASINs: {unique_asin_count}")

Number of unique ASINs: 459006


In [ ]:
import pandas as pd

# Assuming your DataFrame is 'df'
sampled_df_list = []

for cat in df['category'].unique():
    df_cat = df[df['category'] == cat]
    
    # Get unique ASINs in this category
    unique_asins = df_cat['asin'].unique()
    
    # Sample 50k ASINs (or all if less than 50k)
    sampled_asins = pd.Series(unique_asins).sample(n=min(50000, len(unique_asins)), random_state=42)
    
    # Keep all reviews corresponding to these sampled ASINs
    sampled_df_cat = df_cat[df_cat['asin'].isin(sampled_asins)]
    sampled_df_list.append(sampled_df_cat)

# Combine sampled data
sampled_df = pd.concat(sampled_df_list, ignore_index=True)

print(f"Original dataset shape: {df.shape}")
print(f"Sampled dataset shape: {sampled_df.shape}")

Original dataset shape: (3007446, 11)
Sampled dataset shape: (342258, 11)


In [44]:
sampled_df.to_csv("cleaned_dataset4.csv", index=False)